In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

# TEST ZMIANY

# Zadanie 2

In [10]:
# Prawdopodobieństwa, że urządzenie nie ulegnie awarii w zależności od kwoty przeznaczonej na remont
probabilities = {
    'U1': [0.2, 0.2, 0.4, 0.6, 0.7],
    'U2': [0.1, 0.3, 0.5, 0.7, 0.8],
    'U3': [0.2, 0.4, 0.6, 0.8, 0.9],
    'U4': [0.1, 0.2, 0.3, 0.5, 0.7]
}

# Wartości dostępne do wydania na remont w tys. PLN
amounts = [0, 1, 2, 3, 4]  # Kwoty przeznaczone na remonty (0-4 tys. PLN)

# Budżet całkowity
total_budget = 10  # 10 tysięcy PLN

# Funkcja do obliczenia maksymalnego prawdopodobieństwa braku awarii oraz przydziału środków
def max_reliability_with_allocation(total_budget, probabilities):
    num_devices = len(probabilities)  # Liczba urządzeń
    
    # Tablica DP: dp[i][j] oznacza maksymalne prawdopodobieństwo dla i urządzeń i j tysięcy PLN
    dp = np.zeros((num_devices + 1, total_budget + 1))
    
    # Tablica przechowująca optymalny przydział środków
    allocation = np.zeros((num_devices + 1, total_budget + 1), dtype=int)
    
    # Inicjalizacja - dla 0 urządzeń prawdopodobieństwo awarii to 1 (nie ma urządzeń do naprawy)
    dp[0, 0] = 1  # Nie ma urządzeń, więc prawdopodobieństwo braku awarii to 1

    # Programowanie dynamiczne
    for device in range(1, num_devices + 1):  # Przechodzimy po urządzeniach
        for budget in range(total_budget + 1):  # Przechodzimy po dostępnych kwotach
            for spent in amounts:
                if budget - spent >= 0:
                    prob_device = probabilities[f'U{device}'][spent]
                    new_prob = dp[device - 1, budget - spent] * prob_device
                    if new_prob > dp[device, budget]:
                        dp[device, budget] = new_prob
                        allocation[device, budget] = spent

    # Maksymalne prawdopodobieństwo braku awarii całego obiektu
    max_probability = dp[num_devices, total_budget]

    # Odtwarzanie optymalnego przydziału środków
    budget_left = total_budget
    optimal_allocation = []
    for device in range(num_devices, 0, -1):
        spent = allocation[device, budget_left]
        optimal_allocation.append((f'U{device}', spent))
        budget_left -= spent
    
    # Zwracamy maksymalne prawdopodobieństwo oraz przydział środków na urządzenia
    optimal_allocation.reverse()
    return max_probability, optimal_allocation

# Obliczamy maksymalne prawdopodobieństwo i przydział środków
max_probability, optimal_allocation = max_reliability_with_allocation(total_budget, probabilities)

# Wyświetlamy wyniki
print(f'Maksymalne prawdopodobieństwo braku awarii: {max_probability:.4f}')
print("Optymalny przydział środków:")
for device, spent in optimal_allocation:
    print(f'{device}: {spent} tys. PLN')

Maksymalne prawdopodobieństwo braku awarii: 0.0900
Optymalny przydział środków:
U1: 3 tys. PLN
U2: 2 tys. PLN
U3: 2 tys. PLN
U4: 3 tys. PLN


# Zadanie 3

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd

# Parametry
c = 20          # koszt jednostki zakupu
m = 5           # koszt magazynowania jednostki
z = 35          # cena detaliczna
capacity = 20   # pojemność magazynu
T = 30          # liczba etapów

states = np.arange(capacity + 1)  # stany magazynowe 0 ... 20
demands = np.arange(5)            # popyt 0 ... 4
p = 1.0 / len(demands)            # prawdopodobieństwo popytu

# Wartość końcowa: koszt magazynowania resztek
V_next = - (m + z) * states.astype(float)

# Macierze do przechowywania polityki i wartości
policy_matrix = np.zeros((T, capacity + 1), dtype=int)
value_matrix = np.zeros((T + 1, capacity + 1))

# Ustawiamy wartości dla ostatniego etapu
value_matrix[T, :] = V_next

# Algorytm dynamicznego programowania - wsteczna indukcja
for t in range(T - 1, -1, -1):
    V_curr = np.empty_like(V_next)
    policy_t = np.empty_like(states, dtype=int)

    for s in states:
        best_value = -np.inf
        best_x = 0

        # Rozważamy wszystkie możliwe ilości zamówienia
        for x in range(capacity - s + 1):  # x ∈ {0, 1, ..., capacity - s}
            inv = s + x
            expected_value = 0.0

            # Popyt jest losowy, więc obliczamy wartość oczekiwaną
            for d in demands:
                sales = min(inv, d)
                leftover = inv - sales
                immediate_profit = z * sales - c * x - m * leftover
                expected_value += p * (immediate_profit + V_next[leftover])

            if expected_value > best_value:
                best_value = expected_value
                best_x = x

        V_curr[s] = best_value
        policy_t[s] = best_x

    policy_matrix[t, :] = policy_t
    value_matrix[t, :] = V_curr
    V_next = V_curr  # Przechodzimy do poprzedniego etapu

# Konwertujemy do DataFrame, aby ułatwić wyświetlanie
policy_df = pd.DataFrame(
    policy_matrix,
    index=[f"t={t}" for t in range(T)],
    columns=[f"s={s}" for s in states]
)

value_df = pd.DataFrame(
    value_matrix[:-1],  # Zignorujmy ostatni wiersz (etap T)
    index=[f"t={t}" for t in range(T)],
    columns=[f"s={s}" for s in states]
)

# Wyświetlamy optymalną politykę zamówień
import ace_tools as tools; tools.display_dataframe_to_user("Optimal policy (order quantity x_t for each state s_t and stage t)", policy_df)

# Wyświetlamy funkcję wartości
tools.display_dataframe_to_user("Optimal value function V_t(s)", value_df)


ModuleNotFoundError: No module named 'ace_tools'